In [87]:
#from pulp import LpMinimize, LpVariable, LpProblem, LpInteger
from pulp import *

import numpy as np
import itertools

# Problem Definition

### Sudoku Problem using Chromatic Number

A Sudoku problem occurs where there is an incomplete 9x9 table of numbers which must be filled according to the following rules:
- Within any column of the grid, each number between 1 and 9 must appear only once
- Within any row of the grid, each number between 1 and 9 must appear only once
- Dividing the grid into 9 independent 3x3 boxes, each number between 1 and 9 must appear only once per box

The Chromatic Number of a Graph is defined as the minimum number of colours required to colour all vertices in a graph such that no two connected vertices have the same colour. This principal has been used to help solve problems including timetabling, scheduling or, in this case, a Sudoku puzzle.

To help solve the Sudoku problem using Chromatic Number we can think about a Sudoku puzzle as a undirected graph with a Chromatic Number of 9, with each square in the grid being a vertex and edges occurring between numbers from the same <b>row, column and box</b>.

### ILP Formulation

Given the above rules, the Sudoku problem can be defined as an ILP Problem with the following decision variables:

1. Let $X_i \lbrace_{0 \text{ otherwise}}^{1 \text{ if number i is used}}$
2. Let $Y_ij \lbrace_{0 \text{ otherwise}}^{1 \text{ if node j is given number i}}$

Where $i \in \lbrace1,2,3,4,5,6,7,8,9\rbrace$ and $j \in \lbrace1,2,\text{...},79,80\rbrace$

It then follows that the objective function of the problem can be defined as:
<div class="alert alert-block alert-warning">
$min\sum \limits_{i=1}^{9} X_{i}$
<br>
<i>s.t</i>
<br><br>
$\sum \limits_{i=1}^{9} X_{i} = 1 \text{, for each vertex j i.e each cell in grid must be assigned a single number}$
<br>
$Y_ji + Y_wi \leq 1 \text{, for all pairs of connected vertices j}$
<br>
$X_i \geq Y_ij \text{, for each vertex j i.e. number is used when it has been assigned to a vertex}$
<br>
$Y_ij \in {0, 1}$
<br>
$X_i \in {0, 1}$
</div>


In [90]:
def solve_sudoku(input_fill, input_solution=None):
    
    #Create list of nodes in 9x9 grid (from 0 to 80)
    V = list(range(0, 81))
    
    ####Create the edges
    E = []
    
    #1) Vertical edges
    for row in range(0, 81, 9):
        nodes = list(range(row, row+9))
        E += list(itertools.combinations(nodes,2))
    
    
    #2) Hortizontal edges
    for col in range(9):
        nodes = list(range(col, 81, 9))
        E += list(itertools.combinations(nodes,2))
    
    #3) Edges within squares
    '''
    Create a list of list, each list stores all the cell ids that need 
    to relate to each other within squares
    '''
    square = 1
    square_row = 1
    square_row_cell = 1
    index = 0
    square_ids = []

    while square <= 9:
        grand_square_row = 1
        while grand_square_row <= 3:
            temp_lst = []
            square_row_cell = index
            while square_row <= 3:
                temp_lst.append(square_row_cell)
                square_row_cell += 1
                temp_lst.append(square_row_cell)
                square_row_cell += 1
                temp_lst.append(square_row_cell)
                square_row_cell += 7
                square_row += 1
            square_ids.append(temp_lst) 
            square += 1
            index += 3
            square_row = 1
            grand_square_row += 1
        index += 18

    '''
    Stores all combination of each square as an edge. 
    '''
    for lst in square_ids:
        for item in lst:
            for i in range(len(lst)-1):
                if item < lst[i+1]:
                    E.append([item,lst[i+1]])

        
        
    # And the following colors
    colors = {1, 2, 3, 4, 5, 6, 7, 8, 9}

    graph_model = LpProblem('Sudoku Problem', LpMinimize)

    x = LpVariable.dict('x', colors, 0,1,LpInteger)
    graph_model += sum([ x[i] for i in colors])

    '''
    The following part creates a lists of lists,  where each sublist stores all the combinations of variables, for example:
    [['Node: 0 Color: 0',
      'Node: 0 Color: 1',
      'Node: 0 Color: 2',
      'Node: 0 Color: 3',
      'Node: 0 Color: 4',
      'Node: 0 Color: 5',
      'Node: 0 Color: 6',
      'Node: 0 Color: 7',
      'Node: 0 Color: 8',
      'Node: 0 Color: 9'],
     ['Node: 1 Color: 0',...
    Each variable will get the number = 1 when that color (or number in the sudoku) is assigned to that node.
    '''
    Y = []
    for j in V:
        sub_y = []
        for i in colors:
            sub_y.append(str("Node: "+str(j)+ " Color: "+str(i)))
        Y.append(sub_y)

    '''
    The following part creates the previous variables into variables using pulp syntax and sets them as binary integers (taking values 0 or 1).
    It then stores them in a list so that we can create the restrictions for each node in a loop.
    Each restriction states that all the variables for a certain node can sum up to 1 i.e. can only be assigned 1 colour/number 
    '''
    y_0 = LpVariable.dict('y_0_var', [ Y[0][i-1] for i in colors ], 0,1,LpInteger)
    y_1 = LpVariable.dict('y_1_var', [ Y[1][i-1] for i in colors ], 0,1,LpInteger)
    y_2 = LpVariable.dict('y_2_var', [ Y[2][i-1] for i in colors ], 0,1,LpInteger)
    y_3 = LpVariable.dict('y_3_var', [ Y[3][i-1] for i in colors ], 0,1,LpInteger)
    y_4 = LpVariable.dict('y_4_var', [ Y[4][i-1] for i in colors ], 0,1,LpInteger)
    y_5 = LpVariable.dict('y_5_var', [ Y[5][i-1] for i in colors ], 0,1,LpInteger)
    y_6 = LpVariable.dict('y_6_var', [ Y[6][i-1] for i in colors ], 0,1,LpInteger)
    y_7 = LpVariable.dict('y_7_var', [ Y[7][i-1] for i in colors ], 0,1,LpInteger)
    y_8 = LpVariable.dict('y_8_var', [ Y[8][i-1] for i in colors ], 0,1,LpInteger)
    y_9 = LpVariable.dict('y_9_var', [ Y[9][i-1] for i in colors ], 0,1,LpInteger)
    y_10 = LpVariable.dict('y_10_var', [ Y[10][i-1] for i in colors ], 0,1,LpInteger)
    y_11 = LpVariable.dict('y_11_var', [ Y[11][i-1] for i in colors ], 0,1,LpInteger)
    y_12 = LpVariable.dict('y_12_var', [ Y[12][i-1] for i in colors ], 0,1,LpInteger)
    y_13 = LpVariable.dict('y_13_var', [ Y[13][i-1] for i in colors ], 0,1,LpInteger)
    y_14 = LpVariable.dict('y_14_var', [ Y[14][i-1] for i in colors ], 0,1,LpInteger)
    y_15 = LpVariable.dict('y_15_var', [ Y[15][i-1] for i in colors ], 0,1,LpInteger)
    y_16 = LpVariable.dict('y_16_var', [ Y[16][i-1] for i in colors ], 0,1,LpInteger)
    y_17 = LpVariable.dict('y_17_var', [ Y[17][i-1] for i in colors ], 0,1,LpInteger)
    y_18 = LpVariable.dict('y_18_var', [ Y[18][i-1] for i in colors ], 0,1,LpInteger)
    y_19 = LpVariable.dict('y_19_var', [ Y[19][i-1] for i in colors ], 0,1,LpInteger)
    y_20 = LpVariable.dict('y_20_var', [ Y[20][i-1] for i in colors ], 0,1,LpInteger)
    y_21 = LpVariable.dict('y_21_var', [ Y[21][i-1] for i in colors ], 0,1,LpInteger)
    y_22 = LpVariable.dict('y_22_var', [ Y[22][i-1] for i in colors ], 0,1,LpInteger)
    y_23 = LpVariable.dict('y_23_var', [ Y[23][i-1] for i in colors ], 0,1,LpInteger)
    y_24 = LpVariable.dict('y_24_var', [ Y[24][i-1] for i in colors ], 0,1,LpInteger)
    y_25 = LpVariable.dict('y_25_var', [ Y[25][i-1] for i in colors ], 0,1,LpInteger)
    y_26 = LpVariable.dict('y_26_var', [ Y[26][i-1] for i in colors ], 0,1,LpInteger)
    y_27 = LpVariable.dict('y_27_var', [ Y[27][i-1] for i in colors ], 0,1,LpInteger)
    y_28 = LpVariable.dict('y_28_var', [ Y[28][i-1] for i in colors ], 0,1,LpInteger)
    y_29 = LpVariable.dict('y_29_var', [ Y[29][i-1] for i in colors ], 0,1,LpInteger)
    y_30 = LpVariable.dict('y_30_var', [ Y[30][i-1] for i in colors ], 0,1,LpInteger)
    y_31 = LpVariable.dict('y_31_var', [ Y[31][i-1] for i in colors ], 0,1,LpInteger)
    y_32 = LpVariable.dict('y_32_var', [ Y[32][i-1] for i in colors ], 0,1,LpInteger)
    y_33 = LpVariable.dict('y_33_var', [ Y[33][i-1] for i in colors ], 0,1,LpInteger)
    y_34 = LpVariable.dict('y_34_var', [ Y[34][i-1] for i in colors ], 0,1,LpInteger)
    y_35 = LpVariable.dict('y_35_var', [ Y[35][i-1] for i in colors ], 0,1,LpInteger)
    y_36 = LpVariable.dict('y_36_var', [ Y[36][i-1] for i in colors ], 0,1,LpInteger)
    y_37 = LpVariable.dict('y_37_var', [ Y[37][i-1] for i in colors ], 0,1,LpInteger)
    y_38 = LpVariable.dict('y_38_var', [ Y[38][i-1] for i in colors ], 0,1,LpInteger)
    y_39 = LpVariable.dict('y_39_var', [ Y[39][i-1] for i in colors ], 0,1,LpInteger)
    y_40 = LpVariable.dict('y_40_var', [ Y[40][i-1] for i in colors ], 0,1,LpInteger)
    y_41 = LpVariable.dict('y_41_var', [ Y[41][i-1] for i in colors ], 0,1,LpInteger)
    y_42 = LpVariable.dict('y_42_var', [ Y[42][i-1] for i in colors ], 0,1,LpInteger)
    y_43 = LpVariable.dict('y_43_var', [ Y[43][i-1] for i in colors ], 0,1,LpInteger)
    y_44 = LpVariable.dict('y_44_var', [ Y[44][i-1] for i in colors ], 0,1,LpInteger)
    y_45 = LpVariable.dict('y_45_var', [ Y[45][i-1] for i in colors ], 0,1,LpInteger)
    y_46 = LpVariable.dict('y_46_var', [ Y[46][i-1] for i in colors ], 0,1,LpInteger)
    y_47 = LpVariable.dict('y_47_var', [ Y[47][i-1] for i in colors ], 0,1,LpInteger)
    y_48 = LpVariable.dict('y_48_var', [ Y[48][i-1] for i in colors ], 0,1,LpInteger)
    y_49 = LpVariable.dict('y_49_var', [ Y[49][i-1] for i in colors ], 0,1,LpInteger)
    y_50 = LpVariable.dict('y_50_var', [ Y[50][i-1] for i in colors ], 0,1,LpInteger)
    y_51 = LpVariable.dict('y_51_var', [ Y[51][i-1] for i in colors ], 0,1,LpInteger)
    y_52 = LpVariable.dict('y_52_var', [ Y[52][i-1] for i in colors ], 0,1,LpInteger)
    y_53 = LpVariable.dict('y_53var', [ Y[53][i-1] for i in colors ], 0,1,LpInteger)
    y_54 = LpVariable.dict('y_54_var', [ Y[54][i-1] for i in colors ], 0,1,LpInteger)
    y_55 = LpVariable.dict('y_55_var', [ Y[55][i-1] for i in colors ], 0,1,LpInteger)
    y_56 = LpVariable.dict('y_56_var', [ Y[56][i-1] for i in colors ], 0,1,LpInteger)
    y_57 = LpVariable.dict('y_57_var', [ Y[57][i-1] for i in colors ], 0,1,LpInteger)
    y_58 = LpVariable.dict('y_58_var', [ Y[58][i-1] for i in colors ], 0,1,LpInteger)
    y_59 = LpVariable.dict('y_59_var', [ Y[59][i-1] for i in colors ], 0,1,LpInteger)
    y_60 = LpVariable.dict('y_60_var', [ Y[60][i-1] for i in colors ], 0,1,LpInteger)
    y_61 = LpVariable.dict('y_61_var', [ Y[61][i-1] for i in colors ], 0,1,LpInteger)
    y_62 = LpVariable.dict('y_62_var', [ Y[62][i-1] for i in colors ], 0,1,LpInteger)
    y_63 = LpVariable.dict('y_63_var', [ Y[63][i-1] for i in colors ], 0,1,LpInteger)
    y_64 = LpVariable.dict('y_64_var', [ Y[64][i-1] for i in colors ], 0,1,LpInteger)
    y_65 = LpVariable.dict('y_65_var', [ Y[65][i-1] for i in colors ], 0,1,LpInteger)
    y_66 = LpVariable.dict('y_66_var', [ Y[66][i-1] for i in colors ], 0,1,LpInteger)
    y_67 = LpVariable.dict('y_67_var', [ Y[67][i-1] for i in colors ], 0,1,LpInteger)
    y_68 = LpVariable.dict('y_68_var', [ Y[68][i-1] for i in colors ], 0,1,LpInteger)
    y_69 = LpVariable.dict('y_69_var', [ Y[69][i-1] for i in colors ], 0,1,LpInteger)
    y_70 = LpVariable.dict('y_70_var', [ Y[70][i-1] for i in colors ], 0,1,LpInteger)
    y_71 = LpVariable.dict('y_71_var', [ Y[71][i-1] for i in colors ], 0,1,LpInteger)
    y_72 = LpVariable.dict('y_72_var', [ Y[72][i-1] for i in colors ], 0,1,LpInteger)
    y_73 = LpVariable.dict('y_73_var', [ Y[73][i-1] for i in colors ], 0,1,LpInteger)
    y_74 = LpVariable.dict('y_74_var', [ Y[74][i-1] for i in colors ], 0,1,LpInteger)
    y_75 = LpVariable.dict('y_75_var', [ Y[75][i-1] for i in colors ], 0,1,LpInteger)
    y_76 = LpVariable.dict('y_76_var', [ Y[76][i-1] for i in colors ], 0,1,LpInteger)
    y_77 = LpVariable.dict('y_77_var', [ Y[77][i-1] for i in colors ], 0,1,LpInteger)
    y_78 = LpVariable.dict('y_78_var', [ Y[78][i-1] for i in colors ], 0,1,LpInteger)
    y_79 = LpVariable.dict('y_79_var', [ Y[79][i-1] for i in colors ], 0,1,LpInteger)
    y_80 = LpVariable.dict('y_80_var', [ Y[80][i-1] for i in colors ], 0,1,LpInteger)


    y_lst= [y_0, y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9,
           y_10, y_11, y_12, y_13, y_14, y_15, y_16, y_17, y_18, y_19,
           y_20, y_21, y_22, y_23, y_24, y_25, y_26, y_27, y_28, y_29,
           y_30, y_31, y_32, y_33, y_34, y_35, y_36, y_37, y_38, y_39,
           y_40, y_41, y_42, y_43, y_44, y_45, y_46, y_47, y_48, y_49,
           y_50, y_51, y_52, y_53, y_54, y_55, y_56, y_57, y_58, y_59,
           y_60, y_61, y_62, y_63, y_64, y_65, y_66, y_67, y_68, y_69,
           y_70, y_71, y_72, y_73, y_74, y_75, y_76, y_77, y_78, y_79,
           y_80]

    for item in y_lst:
        graph_model += sum([ item[var] for var in item ] ) == 1

    '''
    The following part creates the restriction where for each edge there can only be one color assigned.
    '''
    E_lst = list(E)
    for e in E_lst:
        for i in colors:
            key_0 = str("Node: "+ str(e[0]) + " Color: "+ str(i))
            key_1 = str("Node: "+ str(e[1]) + " Color: "+ str(i))
            graph_model += sum([ y_lst[e[0]][key_0] + y_lst[e[1]][key_1] ] ) <= 1    

    '''
    The following part creates the restrictions where whenever a color is used for
    a node, then that color is activated. 
    '''         
    for j in colors:
        for i in V:
            key = str("Node: "+ str(i) + " Color: "+ str(j))
            graph_model += x[j] >= y_lst[i][key]  
            
    # Pre feed
    
    for k,v in input_fill.items():
        key = str("Node: "+ str(k) + " Color: "+ str(v))
        graph_model += y_lst[k][key] == 1

        
        
    # Pre Add a solution  
    if input_solution is not None:
        for k,v in input_solution.items():
            key = str("Node: "+ str(k) + " Color: "+ str(v))
            graph_model += y_lst[k][key] == 1
        
        
    graph_model.solve()
    #solve_CBC(graph_model, use_mps=True)
    
    solution_dict = {}
    # Print result with the current input:
    if LpStatus[graph_model.status] == 'Optimal':
        i = 0
        result = ""
        while i<=80:
            for j in colors:
                key = str("Node: "+ str(i) + " Color: "+ str(j))
                if y_lst[i][key].value() == 1:
                    if i%9 != 0:
                        result += "| "+ str(j)
                    else: 
                        result += "\n"+ str(j)
            i += 1
            
        solution_dict = {
            "Status": LpStatus[graph_model.status],
            "Result Table": result 
        }

    else:
        solution_dict = {
            "Status": LpStatus[graph_model.status],
            "Result Table": None
        }
    
    return solution_dict
    

In [91]:
def sudoku_solver(input_fill, input_solution = None, show_other_results = False):
    solution_dict = solve_sudoku(input_fill, input_solution)
    
    if solution_dict["Status"] == "Unfeasible":
        print("The current solution is Unfeasible.")
        return 0
    else:
        print("The current solution status is:", solution_dict["Status"])
        print("Result Table: ", solution_dict["Result Table"])
    
    i = 0
    n = 0
    other_results = []
    while i<=80:
        j = 1
        while j<=9:
            if i not in input_fill.keys():
                try_solution = solve_sudoku(input_fill, {i:j})
                if try_solution["Result Table"] == solution_dict["Result Table"]:
                    print("Not trying to find changing node: " + str(i) + " with color: " + str(j) + " since its the current solution.")
                else:
                    
                    print("trying to find changing node: " + str(i) + " with color: " + str(j))
                    if try_solution["Status"] == "Optimal":
                        n+=1
                        if n>1 and not show_other_results and try_solution["Result Table"] != solution_dict["Result Table"]: 
                            print("For the current Sudoku a solution is: \n")
                            for result in other_results:
                                print(result)
                            return 1
                        other_results.append(try_solution["Result Table"])

                        if show_other_results:
                            print("For the current Sudoku there is more than one solution such as: \n")
                            print(try_solution["Result Table"])
            else:
                print("Not trying to find changing node: " + str(i) + " since its part of the pre fill.")
            j += 1
        i += 1
    print("And the current solution is Unique. No other solutions were found.")
    return 0

In [92]:
'''
Here we create a dictionary for the pre-filled cells and pre-solution filled cells (not much of UX implemented here, just need to be hard-coded here).
'''
pre_fill = {2:6, 11:2, 14:4, 17:5, 19:7, 20:5, 23:9, 24:8, 25:3, 28:1, 29:3, 
            31:5, 34:7, 35:8, 41:6, 47:7, 51:9, 57:9, 58:7, 62:4, 65:8, 66:6, 
            71:3, 73:5, 75:8}
pre_solution = None

In [93]:
sudoku_solver(pre_fill, pre_solution, False)

NameError: name 'solve_CBC' is not defined

In [88]:
pulpTestAll()

ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss..........

	 Test that logic put in place for deprecation handling of indexs works
	 Testing 'indexs' param continues to work for LpVariable.dicts
	 Testing 'indexs' param continues to work for LpVariable.matrix
	 Testing 'indices' argument works in LpVariable.dicts
	 Testing 'indices' param continues to work for LpVariable.matrix
	 Testing invalid status
	 Testing continuous LP solution - export dict
	 Testing export dict for LP
	 Testing export dict MIP
	 Testing maximize continuous LP solution
	 Testing continuous LP solution - export JSON
	 Testing continuous LP solution - export solver dict


...............

	 Testing continuous LP solution - export solver JSON
	 Testing reading MPS files - binary variable, no constraint names
	 Testing reading MPS files - integer variable
	 Testing reading MPS files - maximize
	 Testing invalid var names
	 Testing makeDict general behavior
	 Testing makeDict default value behavior
	 Testing measuring optimization time
	 Testing the availability of the function pulpTestAll
	 Testing zero subtraction
	 Testing inconsistent lp solution
	 Testing continuous LP solution


..........

...

..........

	 Testing column based modelling
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)


....ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss

	 Testing elastic constraints (penalty unbounded)
	 Testing timeLimit argument


ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss
----------------------------------------------------------------------
Ran 840 tests in 3.121s

OK (skipped=784)


In [95]:
solver_list = listSolvers(onlyAvailable=True)

In [96]:
print(solver_list)

['GLPK_CMD']
